# Comparación SQL vs NoSQL: Datos de Limpieza de Aulas

Este notebook demuestra las diferencias entre bases de datos relacionales (SQL) y no relacionales (NoSQL) usando como ejemplo los datos de limpieza de aulas y calificaciones de estudiantes.

## 🎯 Analogías Visuales
- **SQL = Balde con agua**: Estructura rígida, forma fija, todos los elementos deben seguir el mismo esquema
- **NoSQL = Hoja de papel**: Estructura libre, cada documento puede tener campos diferentes
- **Tabla SQL = Tabla con columnas**: CAU, Nota, Aseo - esquema predefinido y obligatorio

## 1. Importar Librerías y Configuración

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

# Para reproducibilidad
np.random.seed(42)

print("✅ Librerías importadas correctamente")
print("🎯 Listo para comparar SQL vs NoSQL con datos de aulas")

## 2. Escenario 1: SQL - Estructura Rígida (El Balde)

### 🪣 **Analogía del Balde**
Como un balde que tiene una forma fija, en SQL todos los registros deben seguir **exactamente** el mismo esquema. Si queremos agregar un campo nuevo, debemos modificar TODA la estructura.

**Características SQL:**
- Esquema fijo y predefinido
- Todas las filas tienen las mismas columnas
- Valores nulos si falta información
- Integridad referencial

In [ ]:
# Generar datos sintéticos para la tabla SQL - ESQUEMA FIJO
print("🏗️ Creando tabla SQL con esquema rígido...")
print("📋 Esquema fijo: ID, Aula, Estudiante, Nivel_Aseo, Materiales, Nota_Final, Fecha")

# Datos base para SQL
n_records = 200
aulas = [f"{letra}-{num:03d}" for letra in ['A', 'B', 'C'] for num in range(101, 151)]

# TABLA SQL - ESTRUCTURA RÍGIDA
sql_data = {
    'ID': range(1, n_records + 1),
    'Aula': np.random.choice(aulas, n_records),
    'Estudiante': [f"EST{i:04d}" for i in range(1001, 1001 + n_records)],
    'Nivel_Aseo': np.random.choice(['Excelente', 'Bueno', 'Regular', 'Deficiente'], n_records, p=[0.2, 0.3, 0.3, 0.2]),
    'Materiales': np.random.choice([True, False], n_records, p=[0.7, 0.3]),
    'Nota_Final': np.round(np.random.normal(6.5, 1.5, n_records), 1),
    'Fecha': [datetime.now() - timedelta(days=np.random.randint(1, 30)) for _ in range(n_records)]
}

# Asegurar que las notas estén entre 1 y 10
sql_data['Nota_Final'] = np.clip(sql_data['Nota_Final'], 1.0, 10.0)

# Crear DataFrame SQL
df_sql = pd.DataFrame(sql_data)

print(f"✅ Tabla SQL creada con {len(df_sql)} registros")
print(f"📊 Columnas obligatorias: {list(df_sql.columns)}")
print("\n📋 Primeros 5 registros de la tabla SQL:")
print(df_sql.head())

print("\n🔍 Información de la tabla SQL:")
print(df_sql.info())

### 🚨 Problema del SQL: Agregar Nuevos Campos

**Situación:** Queremos agregar información sobre `Temperatura_Aula` y `Profesor_Encargado` para algunos registros, pero en SQL debemos modificar TODA la tabla.

In [ ]:
print("⚠️ PROBLEMA SQL: Agregando nuevos campos a la tabla existente")
print("🔄 En SQL, debemos alterar TODA la estructura para agregar columnas\n")

# Simulando ALTER TABLE en SQL
print("SQL Query simulado:")
print("ALTER TABLE aulas_limpieza ADD COLUMN Temperatura_Aula DECIMAL(3,1);")
print("ALTER TABLE aulas_limpieza ADD COLUMN Profesor_Encargado VARCHAR(50);")

# Agregar las nuevas columnas (TODAS las filas deben tenerlas)
df_sql_expandido = df_sql.copy()
df_sql_expandido['Temperatura_Aula'] = None  # Muchos valores NULL
df_sql_expandido['Profesor_Encargado'] = None  # Muchos valores NULL

# Solo algunos registros tienen los nuevos datos
indices_con_temp = np.random.choice(df_sql_expandido.index, 50, replace=False)
df_sql_expandido.loc[indices_con_temp, 'Temperatura_Aula'] = np.round(np.random.normal(22, 3, 50), 1)

indices_con_prof = np.random.choice(df_sql_expandido.index, 30, replace=False)
profesores = ['Prof. García', 'Prof. López', 'Prof. Martín', 'Prof. Rodríguez']
df_sql_expandido.loc[indices_con_prof, 'Profesor_Encargado'] = np.random.choice(profesores, 30)

print("\n📊 Tabla SQL después de agregar columnas:")
print(f"Total de registros: {len(df_sql_expandido)}")
print(f"Columnas: {list(df_sql_expandido.columns)}")
print(f"Valores NULL en Temperatura: {df_sql_expandido['Temperatura_Aula'].isnull().sum()}")
print(f"Valores NULL en Profesor: {df_sql_expandido['Profesor_Encargado'].isnull().sum()}")

print("\n📋 Muestra de registros con campos nuevos:")
print(df_sql_expandido[df_sql_expandido['Temperatura_Aula'].notnull()].head(3))

## 3. Escenario 2: NoSQL Heterogéneo - Estructura Libre (La Hoja)

### 📄 **Analogía de la Hoja**
Como una hoja de papel en blanco, en NoSQL cada documento puede tener **campos completamente diferentes**. No hay esquema fijo, cada registro es independiente.

**Características NoSQL Heterogéneo:**
- Sin esquema predefinido
- Cada documento puede tener campos únicos
- Flexibilidad total
- Evolución natural de los datos

In [ ]:
print("📄 Creando colección NoSQL con documentos heterogéneos...")
print("🆓 Sin esquema fijo: cada documento puede tener campos diferentes\n")

# Datos base comunes
aulas_nosql = [f"{letra}-{num:03d}" for letra in ['A', 'B', 'C'] for num in range(101, 151)]
estudiantes_nosql = [f"EST{i:04d}" for i in range(2001, 2151)]

# COLECCIÓN NoSQL - DOCUMENTOS HETEROGÉNEOS
documentos_nosql = []

for i in range(150):
    # Documento base (todos tienen estos campos)
    doc = {
        "_id": f"doc_{i+1:03d}",
        "aula": np.random.choice(aulas_nosql),
        "estudiante": estudiantes_nosql[i],
        "nota": round(np.random.normal(6.0, 1.8), 1)
    }
    
    # Agregar campos opcionales aleatoriamente
    if np.random.random() > 0.3:  # 70% tienen nivel de aseo
        doc["nivel_aseo"] = np.random.choice(["excelente", "bueno", "regular", "deficiente"])
    
    if np.random.random() > 0.5:  # 50% tienen materiales
        doc["materiales_disponibles"] = np.random.choice([True, False])
    
    if np.random.random() > 0.7:  # 30% tienen temperatura
        doc["temperatura"] = round(np.random.normal(22, 3), 1)
    
    if np.random.random() > 0.8:  # 20% tienen profesor
        doc["profesor"] = np.random.choice(["García", "López", "Martín", "Rodríguez"])
    
    if np.random.random() > 0.85:  # 15% tienen horario
        doc["horario"] = np.random.choice(["mañana", "tarde", "noche"])
    
    if np.random.random() > 0.9:  # 10% tienen observaciones
        doc["observaciones"] = np.random.choice([
            "aula muy limpia",
            "necesita ventilación", 
            "faltan materiales",
            "excelente estado"
        ])
    
    if np.random.random() > 0.95:  # 5% tienen datos de limpieza detallados
        doc["limpieza_detalle"] = {
            "ultimo_aseo": f"2025-08-{np.random.randint(1, 9):02d}",
            "productos_usados": np.random.choice(["desinfectante", "detergente", "ambos"]),
            "tiempo_limpieza": np.random.randint(15, 60)
        }
    
    # Asegurar que la nota esté entre 1 y 10
    doc["nota"] = max(1.0, min(10.0, doc["nota"]))
    
    documentos_nosql.append(doc)

print(f"✅ Colección NoSQL creada con {len(documentos_nosql)} documentos")
print("\n📄 Ejemplos de documentos heterogéneos:")

# Mostrar diferentes tipos de documentos
for i, doc in enumerate(documentos_nosql[:5]):
    print(f"\n📋 Documento {i+1} (tiene {len(doc)} campos):")
    print(json.dumps(doc, indent=2, ensure_ascii=False))

### 🎯 Ventaja del NoSQL: Flexibilidad Total

In [ ]:
print("🚀 VENTAJA NoSQL: Agregando nuevos campos sin afectar documentos existentes\n")

# Análisis de la variabilidad de campos
campos_por_documento = [len(doc.keys()) for doc in documentos_nosql]
todos_los_campos = set()
for doc in documentos_nosql:
    todos_los_campos.update(doc.keys())

print(f"📊 Estadísticas de campos por documento:")
print(f"   • Mínimo de campos: {min(campos_por_documento)}")
print(f"   • Máximo de campos: {max(campos_por_documento)}")
print(f"   • Promedio de campos: {np.mean(campos_por_documento):.1f}")
print(f"   • Total de campos únicos: {len(todos_los_campos)}")

print(f"\n🏷️ Todos los campos encontrados:")
for campo in sorted(todos_los_campos):
    count = sum(1 for doc in documentos_nosql if campo in doc)
    porcentaje = (count / len(documentos_nosql)) * 100
    print(f"   • {campo:20} -> {count:3d} docs ({porcentaje:5.1f}%)")

# Agregar un nuevo tipo de documento sin afectar los existentes
nuevo_documento = {
    "_id": "doc_especial_001",
    "aula": "A-999",
    "estudiante": "EST9999",
    "nota": 9.5,
    "tipo_aula": "laboratorio",
    "equipos": ["proyector", "computadoras", "aire_acondicionado"],
    "capacidad": 30,
    "ultima_renovacion": "2024-12-15",
    "responsable_mantenimiento": {
        "nombre": "Juan Pérez",
        "telefono": "555-1234",
        "turno": "nocturno"
    }
}

documentos_nosql.append(nuevo_documento)

print("\n🆕 Nuevo documento agregado sin afectar la estructura existente:")
print(json.dumps(nuevo_documento, indent=2, ensure_ascii=False))

## 4. Escenario 3: NoSQL Homogéneo - Estructura Uniforme Sin Esquema

### 📑 **Analogía de Formularios**
Como formularios que siguen el mismo formato por convención, pero sin ser obligatorio. Todos los documentos tienen los mismos campos, pero el esquema no está forzado por la base de datos.

**Características NoSQL Homogéneo:**
- Sin esquema obligatorio
- Estructura uniforme por convención
- Flexibilidad cuando se necesite
- Mejor para consultas y análisis

In [ ]:
print("📑 Creando colección NoSQL homogénea (estructura uniforme por convención)...")
print("🎯 Todos los documentos siguen el mismo formato, pero sin esquema forzado\n")

# COLECCIÓN NoSQL HOMOGÉNEA - Estructura uniforme por convención
documentos_homogeneos = []

for i in range(180):
    doc = {
        "_id": f"aula_{i+1:03d}",
        "codigo_aula": np.random.choice(aulas_nosql),
        "estudiante_id": f"EST{3000+i:04d}",
        "calificacion": round(np.random.normal(7.0, 1.5), 1),
        "estado_limpieza": np.random.choice(["excelente", "bueno", "regular", "deficiente"], p=[0.25, 0.35, 0.25, 0.15]),
        "materiales_limpieza": np.random.choice([True, False], p=[0.75, 0.25]),
        "temperatura_ambiente": round(np.random.normal(23, 2.5), 1),
        "fecha_evaluacion": f"2025-08-{np.random.randint(1, 9):02d}",
        "turno": np.random.choice(["mañana", "tarde", "noche"], p=[0.5, 0.3, 0.2]),
        "inspector": np.random.choice(["Ana Pérez", "Carlos Luna", "María Silva", "Diego Torres"]),
        "puntuacion_detallada": {
            "orden": round(np.random.uniform(6, 10), 1),
            "ventilacion": round(np.random.uniform(5, 10), 1),
            "mobiliario": round(np.random.uniform(6, 10), 1)
        }
    }
    
    # Asegurar que la calificación esté entre 1 y 10
    doc["calificacion"] = max(1.0, min(10.0, doc["calificacion"]))
    
    documentos_homogeneos.append(doc)

print(f"✅ Colección homogénea creada con {len(documentos_homogeneos)} documentos")
print(f"📋 Todos los documentos tienen exactamente {len(documentos_homogeneos[0])} campos")

print("\n📄 Estructura consistente (primeros 3 documentos):")
for i in range(3):
    print(f"\n📋 Documento {i+1}:")
    print(json.dumps(documentos_homogeneos[i], indent=2, ensure_ascii=False))

## 5. Comparación Visual: SQL vs NoSQL

In [ ]:
# Crear visualizaciones comparativas
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Comparación Visual: SQL vs NoSQL - Datos de Aulas', fontsize=16, fontweight='bold')

# 1. Distribución de notas en SQL
axes[0,0].hist(df_sql['Nota_Final'], bins=15, alpha=0.7, color='skyblue', edgecolor='black', label='SQL Rígido')
axes[0,0].set_title('🪣 SQL: Distribución de Notas\n(Estructura Rígida)')
axes[0,0].set_xlabel('Nota Final')
axes[0,0].set_ylabel('Frecuencia')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# 2. Distribución de notas en NoSQL heterogéneo
notas_heterogeneas = [doc['nota'] for doc in documentos_nosql if 'nota' in doc]
axes[0,1].hist(notas_heterogeneas, bins=15, alpha=0.7, color='lightcoral', edgecolor='black', label='NoSQL Libre')
axes[0,1].set_title('📄 NoSQL Heterogéneo: Distribución de Notas\n(Estructura Libre)')
axes[0,1].set_xlabel('Nota')
axes[0,1].set_ylabel('Frecuencia')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# 3. Distribución de notas en NoSQL homogéneo
notas_homogeneas = [doc['calificacion'] for doc in documentos_homogeneos]
axes[1,0].hist(notas_homogeneas, bins=15, alpha=0.7, color='lightgreen', edgecolor='black', label='NoSQL Uniforme')
axes[1,0].set_title('📑 NoSQL Homogéneo: Distribución de Notas\n(Estructura Uniforme)')
axes[1,0].set_xlabel('Calificación')
axes[1,0].set_ylabel('Frecuencia')
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)

# 4. Comparación de flexibilidad de campos
tipos = ['SQL\n(Rígido)', 'NoSQL\n(Heterogéneo)', 'NoSQL\n(Homogéneo)']
campos_promedio = [
    len(df_sql.columns),
    np.mean([len(doc.keys()) for doc in documentos_nosql]),
    len(documentos_homogeneos[0].keys())
]
flexibilidad = [0, 100, 50]  # Porcentaje de flexibilidad

bars = axes[1,1].bar(tipos, campos_promedio, color=['skyblue', 'lightcoral', 'lightgreen'], alpha=0.8)
axes[1,1].set_title('📊 Promedio de Campos por Registro')
axes[1,1].set_ylabel('Número de Campos')

# Agregar valores en las barras
for bar, valor in zip(bars, campos_promedio):
    axes[1,1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3, 
                   f'{valor:.1f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

## 6. Análisis de Flexibilidad y Variabilidad de Campos

In [ ]:
# Crear visualización de variabilidad de campos
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
fig.suptitle('Variabilidad de Campos por Tipo de Base de Datos', fontsize=16, fontweight='bold')

# 1. SQL - Todos los registros tienen exactamente los mismos campos
sql_campos = [len(df_sql.columns)] * len(df_sql)
axes[0].bar(['Todos los\nregistros'], [len(df_sql.columns)], color='skyblue', alpha=0.8, width=0.5)
axes[0].set_title('🪣 SQL: Estructura Rígida\nTodos tienen exactamente\nlos mismos campos')
axes[0].set_ylabel('Número de Campos')
axes[0].set_ylim(0, 15)
axes[0].text(0, len(df_sql.columns) + 0.3, f'{len(df_sql.columns)}', ha='center', fontweight='bold')

# 2. NoSQL Heterogéneo - Variabilidad de campos
campos_heterogeneos = [len(doc.keys()) for doc in documentos_nosql]
axes[1].hist(campos_heterogeneos, bins=8, color='lightcoral', alpha=0.8, edgecolor='black')
axes[1].set_title('📄 NoSQL Heterogéneo\nCada documento puede\ntener campos diferentes')
axes[1].set_xlabel('Número de Campos por Documento')
axes[1].set_ylabel('Frecuencia')

# 3. NoSQL Homogéneo - Uniformidad por convención
campos_homogeneos = [len(doc.keys()) for doc in documentos_homogeneos]
axes[2].bar(['Todos los\ndocumentos'], [len(documentos_homogeneos[0].keys())], color='lightgreen', alpha=0.8, width=0.5)
axes[2].set_title('📑 NoSQL Homogéneo\nEstructura uniforme\npor convención')
axes[2].set_ylabel('Número de Campos')
axes[2].set_ylim(0, 15)
axes[2].text(0, len(documentos_homogeneos[0].keys()) + 0.3, f'{len(documentos_homogeneos[0].keys())}', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

# Estadísticas de variabilidad
print("📊 ESTADÍSTICAS DE VARIABILIDAD DE CAMPOS:")
print("="*60)
print(f"🪣 SQL (Rígido):")
print(f"   • Campos por registro: {len(df_sql.columns)} (siempre)")
print(f"   • Variabilidad: 0% (esquema fijo)")
print(f"   • Valores NULL permitidos: Sí")

print(f"\n📄 NoSQL Heterogéneo (Libre):")
print(f"   • Campos mínimos: {min(campos_heterogeneos)}")
print(f"   • Campos máximos: {max(campos_heterogeneos)}")
print(f"   • Promedio de campos: {np.mean(campos_heterogeneos):.1f}")
print(f"   • Variabilidad: {np.std(campos_heterogeneos):.1f} (desviación estándar)")
print(f"   • Flexibilidad: 100%")

print(f"\n📑 NoSQL Homogéneo (Uniforme):")
print(f"   • Campos por documento: {len(documentos_homogeneos[0].keys())} (por convención)")
print(f"   • Variabilidad: 0% (uniformidad voluntaria)")
print(f"   • Flexibilidad potencial: 100%")

## 7. Simulación de Consultas y Operaciones

In [ ]:
print("🔍 COMPARACIÓN DE CONSULTAS Y OPERACIONES")
print("="*60)

# 1. Consulta: Promedio de notas por nivel de aseo
print("\n📋 Consulta: Promedio de notas por nivel de aseo\n")

# SQL
print("🪣 SQL Query:")
print("SELECT Nivel_Aseo, AVG(Nota_Final) as Promedio")
print("FROM aulas_limpieza")
print("GROUP BY Nivel_Aseo;")
print("\nResultado SQL:")
resultado_sql = df_sql.groupby('Nivel_Aseo')['Nota_Final'].mean().round(2)
for nivel, promedio in resultado_sql.items():
    print(f"  {nivel:12}: {promedio:.2f}")

# NoSQL Heterogéneo
print("\n📄 NoSQL Heterogéneo Query (pseudocódigo):")
print("db.aulas.aggregate([")
print("  { $match: { nivel_aseo: { $exists: true } } },")
print("  { $group: { _id: '$nivel_aseo', promedio: { $avg: '$nota' } } }")
print("])")
print("\nResultado NoSQL Heterogéneo:")
from collections import defaultdict
grupos_nosql = defaultdict(list)
for doc in documentos_nosql:
    if 'nivel_aseo' in doc and 'nota' in doc:
        grupos_nosql[doc['nivel_aseo']].append(doc['nota'])

for nivel, notas in grupos_nosql.items():
    promedio = np.mean(notas)
    print(f"  {nivel:12}: {promedio:.2f}")

# NoSQL Homogéneo
print("\n📑 NoSQL Homogéneo Query:")
print("db.aulas_homogeneas.aggregate([")
print("  { $group: { _id: '$estado_limpieza', promedio: { $avg: '$calificacion' } } }")
print("])")
print("\nResultado NoSQL Homogéneo:")
grupos_homogeneos = defaultdict(list)
for doc in documentos_homogeneos:
    grupos_homogeneos[doc['estado_limpieza']].append(doc['calificacion'])

for nivel, notas in grupos_homogeneos.items():
    promedio = np.mean(notas)
    print(f"  {nivel:12}: {promedio:.2f}")

## 8. Ventajas y Desventajas: Análisis Comparativo

In [ ]:
# Crear tabla comparativa
import pandas as pd

comparacion = {
    'Aspecto': [
        'Esquema',
        'Flexibilidad',
        'Consistencia',
        'Consultas complejas',
        'Escalabilidad',
        'Integridad de datos',
        'Evolución del esquema',
        'Almacenamiento',
        'Velocidad de desarrollo',
        'Curva de aprendizaje'
    ],
    'SQL (Balde Rígido)': [
        '🔒 Fijo y obligatorio',
        '❌ Muy limitada',
        '✅ Muy alta',
        '✅ Excelente (JOINs)',
        '⚠️ Vertical principalmente',
        '✅ Muy alta (ACID)',
        '❌ Compleja (ALTER TABLE)',
        '⚠️ Puede desperdiciar espacio',
        '⚠️ Planificación necesaria',
        '⚠️ Moderada'
    ],
    'NoSQL Heterogéneo (Hoja Libre)': [
        '🆓 Sin esquema',
        '✅ Total flexibilidad',
        '❌ Variable',
        '⚠️ Limitadas',
        '✅ Horizontal fácil',
        '⚠️ Eventual',
        '✅ Natural y fácil',
        '✅ Eficiente',
        '✅ Muy rápida',
        '✅ Fácil'
    ],
    'NoSQL Homogéneo (Formularios)': [
        '📝 Por convención',
        '✅ Alta cuando se necesite',
        '✅ Alta por diseño',
        '✅ Buenas',
        '✅ Horizontal fácil',
        '✅ Buena',
        '✅ Fácil',
        '✅ Muy eficiente',
        '✅ Rápida',
        '✅ Moderada'
    ]
}

df_comparacion = pd.DataFrame(comparacion)

print("📊 TABLA COMPARATIVA COMPLETA:")
print("="*80)
print(df_comparacion.to_string(index=False))

# Visualización de puntuaciones
fig, ax = plt.subplots(1, 1, figsize=(12, 8))

aspectos = ['Flexibilidad', 'Consistencia', 'Escalabilidad', 'Integridad', 'Evolución', 'Desarrollo']
sql_scores = [2, 9, 6, 9, 3, 6]
nosql_hetero_scores = [10, 4, 9, 5, 10, 9]
nosql_homo_scores = [8, 8, 9, 8, 9, 8]

x = np.arange(len(aspectos))
width = 0.25

bars1 = ax.bar(x - width, sql_scores, width, label='🪣 SQL (Rígido)', color='skyblue', alpha=0.8)
bars2 = ax.bar(x, nosql_hetero_scores, width, label='📄 NoSQL Heterogéneo', color='lightcoral', alpha=0.8)
bars3 = ax.bar(x + width, nosql_homo_scores, width, label='📑 NoSQL Homogéneo', color='lightgreen', alpha=0.8)

ax.set_title('Comparación de Puntuaciones por Aspecto\n(1-10, donde 10 es mejor)', fontsize=14, fontweight='bold')
ax.set_xlabel('Aspectos')
ax.set_ylabel('Puntuación')
ax.set_xticks(x)
ax.set_xticklabels(aspectos, rotation=45)
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_ylim(0, 11)

# Agregar valores en las barras
for bars in [bars1, bars2, bars3]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                f'{height}', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

## 9. Casos de Uso Recomendados

In [ ]:
print("🎯 CASOS DE USO RECOMENDADOS")
print("="*60)

print("\n🪣 SQL (ESTRUCTURA RÍGIDA) - Mejor para:")
print("   ✅ Sistemas bancarios y financieros")
print("   ✅ Inventarios y facturación")
print("   ✅ Sistemas de gestión de personal (nómina)")
print("   ✅ Aplicaciones donde la consistencia es crítica")
print("   ✅ Reportes complejos con múltiples tablas")
print("   ✅ Sistemas de registro académico (notas, materias)")
print("\n   📋 Ejemplo del aula: Sistema de notas oficial del colegio")
print("      Todas las materias DEBEN tener: código, nombre, créditos, nota")

print("\n📄 NoSQL HETEROGÉNEO (ESTRUCTURA LIBRE) - Mejor para:")
print("   ✅ Redes sociales (posts variables)")
print("   ✅ Catálogos de productos diversos")
print("   ✅ Sistemas de logging y monitoreo")
print("   ✅ APIs que reciben datos de múltiples fuentes")
print("   ✅ Prototipado rápido y desarrollo ágil")
print("   ✅ Almacenamiento de configuraciones")
print("\n   📋 Ejemplo del aula: Observaciones libres de inspección")
print("      Cada inspector puede anotar lo que considere relevante")

print("\n📑 NoSQL HOMOGÉNEO (ESTRUCTURA UNIFORME) - Mejor para:")
print("   ✅ Sistemas de análitica y métricas")
print("   ✅ APIs REST con datos estructurados")
print("   ✅ Microservicios con entidades bien definidas")
print("   ✅ Aplicaciones móviles con sincronización")
print("   ✅ Sistemas que necesitan escalar horizontalmente")
print("   ✅ Data lakes con estructura conocida")
print("\n   📋 Ejemplo del aula: Evaluaciones estándar de limpieza")
print("      Formato común pero flexible para futuras mejoras")

# Crear matriz de decisión
print("\n\n🤔 MATRIZ DE DECISIÓN:")
print("="*60)

decision_matrix = {
    'Necesidad': [
        'Esquema muy estable',
        'Transacciones ACID',
        'Consultas complejas',
        'Prototipado rápido',
        'Datos muy variables',
        'Escalabilidad horizontal',
        'Desarrollo ágil',
        'Análisis y reportes',
        'Flexibilidad futura'
    ],
    'Recomendación': [
        '🪣 SQL',
        '🪣 SQL',
        '🪣 SQL',
        '📄 NoSQL Heterogéneo',
        '📄 NoSQL Heterogéneo',
        '📑 NoSQL Homogéneo',
        '📄 NoSQL Heterogéneo',
        '📑 NoSQL Homogéneo',
        '📑 NoSQL Homogéneo'
    ]
}

df_decision = pd.DataFrame(decision_matrix)
print(df_decision.to_string(index=False))

## 10. Conclusiones y Reflexión Final

### 🎭 La Analogía Completa: Balde, Hoja y Formulario

In [ ]:
print("🎭 REFLEXIÓN FINAL: LA ANALOGÍA COMPLETA")
print("="*70)

print("\n🪣 EL BALDE (SQL):")
print("   Como un balde de agua, tiene una forma fija y rígida.")
print("   • Todo lo que entra debe adaptarse a esa forma")
print("   • Es muy bueno para contener y organizar")
print("   • Pero no puedes cambiar la forma sin vaciar todo")
print("   • Perfecto cuando sabes exactamente qué vas a almacenar")

print("\n📄 LA HOJA EN BLANCO (NoSQL Heterogéneo):")
print("   Como una hoja de papel, puedes escribir lo que quieras.")
print("   • Cada 'documento' puede ser completamente diferente")
print("   • Máxima libertad creativa")
print("   • Pero puede volverse caótico sin organización")
print("   • Perfecto para datos impredecibles o evolutivos")

print("\n📑 EL FORMULARIO (NoSQL Homogéneo):")
print("   Como formularios que siguen un formato por convención.")
print("   • Estructura consistente pero no obligatoria")
print("   • Fácil de leer y procesar")
print("   • Flexibilidad cuando la necesites")
print("   • Perfecto para la mayoría de aplicaciones modernas")

# Estadísticas finales del análisis
print("\n\n📊 ESTADÍSTICAS FINALES DEL ANÁLISIS:")
print("="*50)
print(f"📋 Registros SQL analizados: {len(df_sql)}")
print(f"📄 Documentos NoSQL heterogéneos: {len(documentos_nosql)}")
print(f"📑 Documentos NoSQL homogéneos: {len(documentos_homogeneos)}")
print(f"🎯 Total de datos procesados: {len(df_sql) + len(documentos_nosql) + len(documentos_homogeneos)}")

print("\n🏆 CONCLUSIÓN PRINCIPAL:")
print("   No hay una solución única para todos los problemas.")
print("   La elección depende del contexto, los requisitos")
print("   y la evolución esperada de los datos.")
print("\n   🎯 La clave está en entender qué tipo de 'contenedor'")
print("      necesitas: ¿un balde, una hoja o un formulario?")

print("\n" + "="*70)
print("🎉 ANÁLISIS COMPLETADO")
print("   Has visto cómo los mismos datos de limpieza de aulas")
print("   se comportan de manera diferente en cada modelo.")
print("   ¡Ahora puedes elegir la herramienta correcta!")
print("="*70)